In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, AdaBoostRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

/Users/eddieowens/.local/share/virtualenvs/ngc-templates-KRpFoM6y/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/eddieowens/.local/share/virtualenvs/ngc-templates-KRpFoM6y/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Users/eddieowens/.local/share/virtualenvs/ngc-templates-KRpFoM6y/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/eddieowens/.local/share/virtualenvs/ngc-templates-KRpFoM6y/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
boxscores = pd.read_csv('wnba_box_scores.csv', index_col=0)

In [4]:
players = pd.read_csv('wnba_player_metadata.csv', index_col=0)

In [10]:
# Calc team stats by game
# Calc player advanced stats by game

In [18]:
boxscores.head()

,Player,MP,FG,FGA,3P,3PA,FT,FTA,ORB,TRB,...,PTS,team,opp,home_away,starter,playoffs,game_id,date,player_link,player_id
0,Cynthia Cooper,36:59,9.0,18.0,0.0,3.0,7.0,8.0,1.0,8.0,...,25.0,Cleveland Rockers,Houston Comets,1,1,True,199706210CLE,1997-06-21,/wnba/players/c/coopecy01w.html,coopecy01w
1,Tina Thompson,34:26,6.0,9.0,2.0,4.0,0.0,0.0,0.0,5.0,...,14.0,Cleveland Rockers,Houston Comets,1,1,True,199706210CLE,1997-06-21,/wnba/players/t/thompti01w.html,thompti01w
2,Janeth Arcain,27:54,8.0,12.0,0.0,0.0,0.0,0.0,1.0,1.0,...,16.0,Cleveland Rockers,Houston Comets,1,1,True,199706210CLE,1997-06-21,/wnba/players/a/arcaija01w.html,arcaija01w
3,Wanda Guyton,20:24,1.0,2.0,0.0,0.0,2.0,2.0,1.0,3.0,...,4.0,Cleveland Rockers,Houston Comets,1,1,True,199706210CLE,1997-06-21,/wnba/players/g/guytowa01w.html,guytowa01w
4,Kim Perrot,20:22,1.0,5.0,0.0,3.0,0.0,0.0,1.0,3.0,...,2.0,Cleveland Rockers,Houston Comets,1,1,True,199706210CLE,1997-06-21,/wnba/players/p/perroki01w.html,perroki01w


In [23]:
tm_stats = boxscores.groupby(['game_id', 'team'])[['MP', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']].sum()

In [24]:
tm_stats['2PA'] = tm_stats['FGA'] - tm_stats['3PA']
tm_stats['2P'] = tm_stats['FG'] - tm_stats['3P']

In [ ]:
# 0.5 * ((Tm FGA + 0.4 * Tm FTA – 1.07 * (Tm ORB / (Tm ORB + Opp DRB)) * (Tm FGA – Tm FG) + Tm TOV) + (Opp FGA + 0.4 * Opp FTA – 1.07 * (Opp ORB / (Opp ORB + Tm DRB)) * (Opp FGA – Opp FG) + Opp TOV))

In [47]:
def calc_gm_poss(gm):
    tm1_name, tm2_name = gm.index
    tm1 = gm.loc[tm1_name]
    tm2 = gm.loc[tm2_name]
    
    return (tm1['FGA'] + 0.4*tm1['FTA'] - 1.07*(tm1['ORB']/(tm1['ORB'] + (tm2['TRB'] - tm2['ORB']))) * \
            (tm1['FGA'] - tm1['FG']) + tm1['TOV'] + tm2['FGA'] + 0.4*tm2['FTA'] - \
            1.07*(tm2['ORB']/(tm2['ORB'] + (tm1['TRB'] - tm1['ORB']))) * (tm2['FGA'] - tm2['FG']) + tm2['TOV'])*.5

In [53]:
for game_id in tm_stats.index.levels[0]:
    gm = tm_stats.loc[game_id]
    tm_stats.loc[game_id, 'poss'] = calc_gm_poss(gm)

In [21]:
def convert_mp_decimal(MP):
    mins, secs = MP.split(':')
    return int(mins) + int(secs)/60

In [22]:
boxscores['2PA'] = boxscores['FGA'] - boxscores['3PA']
boxscores['2P'] = boxscores['FG'] - boxscores['3P']
boxscores['MP'] = boxscores.MP.apply(convert_mp_decimal)

In [9]:
# stats to predict: 2pt pct, 3pt pct, FT pct, FTA/FGA, OREB/Tm FG missed, DREB/opp FG missed, AST/Teammates FG, 
# Blk/Opponent 2pt FGA, Stl/Possessions

In [55]:
for i, row in boxscores.iterrows():
    boxscores.loc[i, 'oReb_chances'] = (tm_stats.loc[(row['game_id'], row['team'])]['FGA'] - \
                                        tm_stats.loc[(row['game_id'], row['team'])]['FG'] + \
                                        tm_stats.loc[(row['game_id'], row['team'])]['FTA'] - \
                                        tm_stats.loc[(row['game_id'], row['team'])]['FT']) * \
                                        row['MP']/(tm_stats.loc[(row['game_id'], row['team'])]['MP']/5)
    boxscores.loc[i, 'dReb_chances'] = (tm_stats.loc[(row['game_id'], row['opp'])]['FGA'] - \
                                        tm_stats.loc[(row['game_id'], row['opp'])]['FG'] + \
                                        tm_stats.loc[(row['game_id'], row['opp'])]['FTA'] - \
                                        tm_stats.loc[(row['game_id'], row['opp'])]['FT']) * \
                                        row['MP']/(tm_stats.loc[(row['game_id'], row['team'])]['MP']/5)
    boxscores.loc[i, 'teammate_fg'] = (tm_stats.loc[(row['game_id'], row['team'])]['FG'] - row['FG']) * \
                                        row['MP']/(tm_stats.loc[(row['game_id'], row['team'])]['MP']/5)
    boxscores.loc[i, 'opp_2fga'] = tm_stats.loc[(row['game_id'], row['opp'])]['2PA'] * \
                                        row['MP']/(tm_stats.loc[(row['game_id'], row['team'])]['MP']/5)
    boxscores.loc[i, 'poss'] = tm_stats.loc[(row['game_id'], row['team'])]['poss'] * \
                                        row['MP']/(tm_stats.loc[(row['game_id'], row['team'])]['MP']/5)

In [56]:
boxscores.head()

,Player,MP,FG,FGA,3P,3PA,FT,FTA,ORB,TRB,...,date,player_link,player_id,2PA,2P,oReb_chances,dReb_chances,teammate_fg,opp_2fga,poss
0,Cynthia Cooper,36.983333,9.0,18.0,0.0,3.0,7.0,8.0,1.0,8.0,...,1997-06-21,/wnba/players/c/coopecy01w.html,coopecy01w,15.0,9.0,24.039167,39.757083,19.416250,49.002917,68.836453
1,Tina Thompson,34.433333,6.0,9.0,2.0,4.0,0.0,0.0,0.0,5.0,...,1997-06-21,/wnba/players/t/thompti01w.html,thompti01w,5.0,4.0,22.381667,37.015833,20.660000,45.624167,64.090181
2,Janeth Arcain,27.900000,8.0,12.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1997-06-21,/wnba/players/a/arcaija01w.html,arcaija01w,12.0,8.0,18.135000,29.992500,15.345000,36.967500,51.929798
3,Wanda Guyton,20.400000,1.0,2.0,0.0,0.0,2.0,2.0,1.0,3.0,...,1997-06-21,/wnba/players/g/guytowa01w.html,guytowa01w,2.0,1.0,13.260000,21.930000,14.790000,27.030000,37.970175
4,Kim Perrot,20.366667,1.0,5.0,0.0,3.0,0.0,0.0,1.0,3.0,...,1997-06-21,/wnba/players/p/perroki01w.html,perroki01w,2.0,1.0,13.238333,21.894167,14.765833,26.985833,37.908132


In [57]:
boxscores.to_csv('wnba_box_scores.csv')

In [5]:
players.head()

,birthdate,college,draft_pos,draft_year,height,player_id,position,weight,country_fixed,name
0,1963-04-14,USC,NaN,NaN,5.833333,coopecy01w,Guard,150.0,us,Cynthia Cooper
1,1975-02-10,USC,1.0,1997.0,6.166667,thompti01w,Forward,178.0,us,Tina Thompson
2,1969-04-11,NaN,NaN,NaN,5.916667,arcaija01w,Guard,147.0,br,Janeth Arcain
3,1968-10-14,South Florida,NaN,NaN,6.083333,guytowa01w,Center,178.0,us,Wanda Guyton
4,1967-01-18,SW Louisiana,NaN,NaN,5.416667,perroki01w,Guard,130.0,us,Kim Perrot
